# 코랩 용 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !gdown https://drive.google.com/u/0/uc?id=1AWOO1awvSGHHKbydWJTmeZ0g5f5rV85I&export=download
# !unzip open.zip
!unzip open/train.zip 
!unzip open/test.zip
!mv train ./open
!mv test ./open 

# 모듈 

In [1]:
import os 
from glob import glob 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn 
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import transforms as T 
import cv2 
import torch.nn.functional as F
from tqdm import tqdm 

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dir_default = './data/open/'
#코랩용 
# dir_default = './open'

def file_load(dir_default):
    dir_train_imgs = sorted(glob(os.path.join(dir_default,'train','*')))
    dir_test_imgs = sorted(glob(os.path.join(dir_default,'test','*')))
    train_df = pd.read_csv(os.path.join(dir_default,'train_df.csv'))
    test_df = pd.read_csv(os.path.join(dir_default,'test_df.csv'))
    return dir_train_imgs,dir_test_imgs,train_df, test_df 

# 데이터셋 

In [8]:
class Dset(Dataset):
    def __init__(self,dir_imgs,labels):
        self.dir_imgs = dir_imgs 
        self.labels = labels 
        self.transform = self.transform_img()

    def __len__(self):
        return len(self.dir_imgs)

    def transform_img(self,train=True):
        if train:
            transform = T.Compose([
                T.ToTensor(),
                T.Resize(256),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                    )
            ])
        return transform 

    def load_img(self,dir_img):
        temp_img = cv2.imread(dir_img)
        temp_img = self.transform(temp_img)
        return temp_img 

    def __getitem__(self,idx):
        img = self.load_img(self.dir_imgs[idx])
        label = label_encoder[self.labels[idx]]
        return img, label 

# 모델 구성 

64.0

In [154]:
class clf(nn.Module):
    def __init__(self):
        super(clf, self).__init__()
        self.layers = nn.Sequential(
            self.conv_block(3,6),
            self.conv_block(6,16),
            self.conv_block(16,64),
            self.conv_block(64,128)
        )
        self.fc1 = nn.Linear(128*16*16,2048)
        self.fc2 = nn.Linear(2048,512)
        self.fc3 = nn.Linear(512,88)


    def conv_block(self,in_c,out_c):
      block = nn.Sequential(
          nn.Conv2d(in_c,out_c,3,padding='same'),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.BatchNorm2d(out_c)
      )
      return block 
      
    def forward(self,x):
      x = self.layers(x)  
      x = nn.Flatten()(x)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = F.relu(self.fc3(x))
      # x = F.softmax(x)
      return x



In [155]:
model = clf().to(device)
model(img).shape

torch.Size([16, 88])

# 학습 

In [156]:
# 데이터 로드 
dir_default = './data/open/'
dir_train_imgs,dir_test_imgs,train_df, test_df  = file_load(dir_default)

#라벨 생성 
train_labels = train_df['label'].to_numpy()
#라벨 인코더 
label_encoder = {b:a for a,b in enumerate(np.unique(train_labels))}
label_decoder = {b:a for a,b in enumerate(np.unique(train_labels))}

#데이터 셋 
train_dset = Dset(dir_train_imgs,train_labels)
train_loader = DataLoader(train_dset,batch_size=16,shuffle=True)

#모델 및 컴파일 
device = 'cuda'
model = clf().to(device) 
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
criterion = nn.CrossEntropyLoss()


In [157]:
for epoch in range(3):
  train_loss = 0.0 
  print(f'total it : {train_loader.__len__()}')
  for i, data in tqdm(enumerate(train_loader)):
    img,labels = data 
    img = img.to(device)
    labels = labels.to(device)
    
    
    #옵티마이저 초기화 
    optimizer.zero_grad() 

    #backward 
    outputs = model(img)
    

    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item() 
    
  print(f'Epoch: {epoch}, train_loss : {train_loss}')    

total it : 268


268it [01:42,  2.61it/s]


Epoch: 0, train_loss : 1212.902249097824
total it : 268


268it [01:42,  2.61it/s]


Epoch: 1, train_loss : 1199.926284790039
total it : 268


268it [01:42,  2.62it/s]

Epoch: 2, train_loss : 1199.926284790039


In [ ]:
model = clf()
model.to(device)

In [138]:
model(img).shape

torch.Size([16, 88])